*This shell script allow [Crawler IP Detect](Crawler IP Detect.ipynb) to do host query concurrently*

```bash
#!/bin/sh

THREAD_NUM=100

# create a PIPE and associate it with file descriptor 9
mkfifo PIPE
exec 9<>PIPE

# echo THREAD_NUM "\n"s to the PIPE to allow THREAD_NUM processes to run initially
for ((i=0; i<$THREAD_NUM; i++))
do
    echo -ne "\n" 1>&9
done

if [ $# != 2 ] ;then
    echo "The parameters you enter is not correct!"
    exit -1
fi

while read ip 
do
{
    # acquire a "\n" from PIPE to run. 
    # if no "\n" available in the PIPE the process will be blocked permanetly
    read -u 9
    {
        output=$(host $ip)
        echo $ip, $output

        # release a "\n" to allow another process to run
        echo -ne "\n" 1>&9
    }&
}
done < $1 >$2

# wait until all subprocesses to finish
wait; echo "Finished."

rm -f PIPE
```